## Singular Value Decomposition
**Singular value decomposition** is a generalization of the eigendecomposition of normal square matrices into rotational and stretching parts to *non-square* matrices. The matrices in the decomposition now also have different dimensions. Assuming $\mathbf A \in \mathbb{R}^{m\times n}$, we get

\begin{equation*} 
  \mathbf A = \mathbf U\mathbf S\mathbf V^T 
\end{equation*}

where $\mathbf U \in \mathbb{R}^{m\times m}$ and orthogonal, $\mathbf S \in \mathbb{R}^{m\times n}$ and diagonal, and $\mathbf V \in \mathbb{R}^{n \times n}$ and orthogonal. So $\mathbf U$ and $\mathbf V$ are rotations, while $\mathbf S$ purely expresses stretching. All non-diagonal elements of $\mathbf S$ are zero. 

The columns of $\mathbf U$ are eigenvectors of $\mathbf A\mathbf A^T$, called *left-singular vectors*. They are real, since $\mathbf U$ is orthogonal. $\mathbf V$ contains the eigenvectors of $\mathbf A^T \mathbf A$, called the *right-singular eigenvectors*. The diagonal of $\mathbf S$ consists of the square roots of the eigenvalues of $\mathbf A^T \mathbf A$, called the *singular values*:

\begin{equation*}
  \text{diag}\left(\mathbf S\right) = \sqrt{\lambda\left(\mathbf A^T \mathbf A\right)}
\end{equation*}

The interpretation of what is happening with this decomposition is very similar to what's happening in an eigendecomposition. $\mathbf V^T$ rotates into the eigenspace of $\mathbf A$, $\mathbf S$ applies stretch factors, $\mathbf U$ rotates the system back. We'll visualize this behavior in the exercises today.

If we spell out the vectorwise multiplications like we did for the eigendecomposition, we get:

\begin{align*}
  A &= \sigma_1 u_1 v_1^\text{T} + \lambda_2 u_2 v_2^\text{T} + \cdots \\
    &= \sigma_1 \begin{bmatrix} \rule[-1ex]{0.5pt}{2.5ex} \\ u_1 \\ \rule[-1ex]{0.5pt}{2.5ex} \end{bmatrix} \begin{matrix}\begin{bmatrix} -- & v_1 & -- \end{bmatrix} \\ \mbox{} \\ \mbox{}\end{matrix} + 
    \lambda_2 \begin{bmatrix} \rule[-1ex]{0.5pt}{2.5ex} \\ u_2 \\ \rule[-1ex]{0.5pt}{2.5ex} \end{bmatrix} \begin{matrix}\begin{bmatrix} -- & v_2 & -- \end{bmatrix} \\ \mbox{} \\ \mbox{}\end{matrix} + \cdots \\
    &= \sum_i^m \sigma_i u_i v_i^\text{T}
\end{align*}

where this time the sum only goes up to the first dimension of the original matrix $\mathbf A$ (can you see why?). This is especially interesting when $n \gg m$, which would be the case when samples are sparse and features are many in a data matrix.

There are several interesting applications. Very often, SVD is used for data compression, where dimensions of the system with small singular values are completely omitted, especially when the SVD of a *covariance matrix* is performed. This can reduce the order of a model significantly. In control theory, SVD is used for developing robust control. It can also be used as a tool to diagonalize matrices, or for *image/data compression*.<br />
The idea for all of these is to take the decomposition $\mathbf U, \mathbf S, \mathbf V$ and *truncate* their axis such that only a specific number of singular values is used. For $\mathbf U$, this would be the second axis, for $\mathbf S$ both axes, and for $\mathbf V$ the first axis.<br />
For the spectral decomposition above, this would be equivalent to cutting off all terms after some index.

Let's see what happens when we use SVD on an image. How many singular values do you need to be able to interpret the image? What's at the bottom right of the image?

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from skimage import data, img_as_float
from skimage.color import rgb2gray

grayscale = True

# taking an astronaut image from scikit-image as an example
image = img_as_float(data.astronaut())
if grayscale:
    image = rgb2gray(image)

# save this for later reshaping back
shape_original = image.shape

# get a 2d matrix of the image, so we can do SVD on it
# this is necessary because it contains 3 channels for all colors
if grayscale:
    image_2d = image
else:
    image_2d = image.reshape(shape_original[0], 3*shape_original[1])


# the SVD
U, S, V = np.linalg.svd(image_2d, full_matrices=False)

def compress_rgb_image(U,S,V, shape_original, cutoff=10):
    compressed_image = U[:,:cutoff] @ np.diag(S[:cutoff]) @ V[:cutoff,:]
    print(U[:,:cutoff].shape, np.diag(S[:cutoff]).shape, V[:cutoff,:].shape)
    
    # get back original shape
    image = compressed_image.reshape(shape_original)
    
    fig, axes = plt.subplots(1, 2, figsize=(12,7))
    
    axes[0].plot(S[:cutoff], lw=4)
    axes[0].set_title(label="all singular values used")
    axes[1].imshow((image * 255).astype(np.uint8), cmap='gray')
    
    plt.tight_layout()
    

interact(compress_rgb_image, 
         U=fixed(U), 
         S=fixed(S), 
         V=fixed(V), 
         shape_original=fixed(shape_original), 
         cutoff=widgets.IntSlider(min=0, max=150, value=2, continuous_update=False));

interactive(children=(IntSlider(value=2, continuous_update=False, description='cutoff', max=150), Output()), _…

We see that not all singular values are necessary to convey meaning in the image. Depending on the level of detail desired, even a few suffice to recognize the astronaut, the US flag and the space ship. These kinds of data compression can be useful to reduce the order of a model in general, not only regarding ML techniques. 

One area where this can be a great way to compress information is computational fluid dynamics. Consider a 2D flow field, the dynamics of which are calculated over some time range $[t_0, \dots, t_m]$, discretized into $m$ time slices, reminiscent of frames from a video. Each 2D slice can be unpacked into a vector, where each entry would be the value of some field quantity at a node (or a pixel for image data) in the system. These slices are the $m$ samples in the data matrix, and the nodal values are the $n$ features. An SVD performed on this data matrix will find $m$ "eigenmodes" of the flow field and express the original flow fields as linear combinations of these eigenmodes. The eigenmodes are the row vectors of $\mathbf U$, the coefficients are the corresponding singular values and entries from $\mathbf V$.<br />
Truncating this description would yield flow fields with lower resolution as approximations to the real flow field.